# 前提条件: サンプルエージェントの作成

## 概要

まず、評価対象となるエージェントを作成しましょう。このチュートリアルでは、異なるフレームワークを使用して評価用の2つのサンプルエージェントを作成します:
- [Strands Agents SDK](https://strandsagents.com/)
- [LangGraph](https://www.langchain.com/langgraph)

両エージェントは Amazon Bedrock の Anthropic Claude Haiku 4.5 を LLM モデルとして使用しますが、お好みのモデルを使用することも可能です。両エージェントは同じ機能を持っています:
- **Math Tool**: 基本的な数学演算を実行するツール
- **Weather Tool**: 天気情報のダミー実装


アーキテクチャは以下のようになります:

![Architecture](../images/agent_architecture.png)

## 前提条件
- Python 3.10以上
- AWS 認証情報

In [ ]:
!pip install -r ../requirements.txt

## セットアップ

必要なパッケージをインポートし、AWS セッションを設定します:

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import uuid
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using region: {region}")

## Strands エージェントのデプロイ
Strands エージェントを AgentCore Runtime にデプロイしましょう

In [ ]:
agentcore_runtime = Runtime()

agent_name = "ac_eval_strands2"
response = agentcore_runtime.configure(
    entrypoint="eval_agent_strands.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements_strands.txt",
    region=region,
    agent_name=agent_name,
    idle_timeout=120
)
launch_result_strands = agentcore_runtime.launch()
print(f"Strands agent deployment started: {launch_result_strands}")

### AgentCore Runtime でのデプロイ状態を確認
デプロイが ACTIVE 状態になるまで待機します..

In [ ]:
def wait_for_deployment(runtime, name):
    end_statuses = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
    while True:
        status_response = runtime.status()
        status = status_response.endpoint['status']
        print(f"{name} status: {status}")
        if status in end_statuses:
            print(f"{name} deployment completed with status: {status}")
            return status
        time.sleep(10)

strands_status = wait_for_deployment(agentcore_runtime, "Strands")

### Runtime 上の Strands エージェントを呼び出す
ペイロードを使用して AgentCore Runtime エンドポイントを呼び出し、Strands エージェントをテストしましょう。

In [ ]:
session_id_strands = str(uuid.uuid4())
print(f"Session ID: {session_id_strands}")

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "How much is 2+2?"},
    session_id=session_id_strands
)
invoke_response

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "How is the weather now?"},
    session_id=session_id_strands
)
invoke_response

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "Can you tell me the capital of the US?"},
    session_id=session_id_strands
)
invoke_response

## LangGraph エージェントを AgentCore Runtime にデプロイ

LangGraph エージェントも AgentCore Runtime にデプロイしましょう。

In [ ]:
langgraph_agentcore_runtime = Runtime()
agent_name = "ac_eval_langgraph2"
response = langgraph_agentcore_runtime.configure(
    entrypoint="eval_agent_langgraph.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements_langgraph.txt",
    region=region,
    agent_name=agent_name,
    idle_timeout=120
)
launch_result_langgraph = langgraph_agentcore_runtime.launch()
print(f"LangGraph agent deployment started: {launch_result_langgraph}")

### LangGraph エージェントの状態を確認
LangGraph エージェントを AgentCore Runtime にデプロイしたので、デプロイ状態を確認しましょう

In [ ]:
langgraph_status = wait_for_deployment(langgraph_agentcore_runtime, "LangGraph")

### Runtime 上の LangGraph エージェントを呼び出す
ペイロードを使用して AgentCore Runtime 上の LangGraph エージェントエンドポイントをテストします:

In [ ]:
session_id_langgraph = str(uuid.uuid4())
print(f"Session ID: {session_id_langgraph}")

In [ ]:
response = langgraph_agentcore_runtime.invoke(
    payload={"prompt": "What is 2+2?"},
    session_id=session_id_langgraph
)
print(response)

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "What is the weather now?"},
    session_id=session_id_langgraph
)
invoke_response

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "Can you tell me the capital of the US?"},
    session_id=session_id_langgraph
)
invoke_response

In [ ]:
print(f"Strands: {launch_result_strands}, Session: {session_id_strands}")
print(f"LangGraph: {launch_result_langgraph}, Session: {session_id_langgraph}")

In [ ]:
%store launch_result_strands
%store session_id_strands
%store launch_result_langgraph
%store session_id_langgraph

## 次のステップ

必要な前提条件がすべて揃いました。それでは個別の評価チュートリアルを進めましょう:
評価チュートリアルを続けます:
- [01-creating-custom-evaluators](../01-creating-custom-evaluators/): カスタム評価メトリクスの作成
- [02-running-evaluations](../02-running-evaluations/): オンデマンドおよびオンライン評価の実行
- [03-advanced](../03-advanced/): 高度なテクニックとダッシュボード